In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

### ввод
'''
мб нужен тестовый сет
'''

df= pd.DataFrame()
df['A'] = np.random.randint(0,50000, size=40000)
df['B'] = np.random.randint(0,50, size=40000)
df['C'] = np.random.randint(0,60, size=40000)
df['D'] = np.random.randint(0,75, size=40000)
df['AA'] = np.random.randint(0,5, size=40000)
df['BB'] = np.random.randint(0,15, size=40000)
df['CC'] = np.random.randint(0,3785, size=40000)
df['DD'] = np.random.randint(0,95, size=40000)
df['EE'] = np.random.randint(0,8, size=40000)
df['E'] = np.random.randint(0,115, size=40000)
df['Target'] = np.random.randint(0,100, size=40000)



### нормируем
def is_binary(x):
    return x.unique().shape[0] == 2


def normalize(x):
    return (x - x.mean()) / x.std()


bin_free = df.columns[~df.apply(is_binary)]
df[bin_free] = df[bin_free].apply(normalize, axis=0)
df['w0_reg_constant'] = 1

#на всякий вырезал выбросы
df = df.drop(df[df.Target > df.Target.quantile(.95)].index)

### перемешиваем
df = shuffle(df)


# функция градиентного спуска
# где new_w задает измение веса с помощью градиентного спуска
def gradient_descent(X, y, theta=1e-2, epsilon=1e-3):

    w = np.random.normal(size=X.shape[1])
    new_w = 1000*w
    N = X.shape[0]
    iter_num = 0

    while np.linalg.norm(w-new_w) >= epsilon:
        new_w = w
        dw = (2 / N) * (X.T.dot( X.dot (w)) - X.T.dot(y))
        w = w - theta*dw
        
        iter_num+=1
        if iter_num > 1000:
            return w
        
    return w


# метрики и прочее
def R2(x, y):
    return 1 - np.sum(np.power(y - x, 2)) / np.sum(np.power(y - y.mean(), 2))

def reg_prediction(X, w):
    return X.dot( w)

def RMSE(x, y):
    return np.sqrt(np.sum(np.power(y - x, 2)) / y.shape[0])


# делим на фолды и мутим свое грязное дело
# вторые копии для наглядости
folds_index = 5
fold_size = round(df.shape[0] / folds_index)

features = pd.DataFrame()
RMSE_test = []
RMSE_train = []
R2_test = []
R2_train = []

for i in range(folds_index):
    test = df[i * fold_size:(i + 1) * fold_size]
    if i == 0:
        train = df[(i + 1) * fold_size:]
    else:
        train = df[:i * fold_size]
        if i != 4:
            train = train.append(df[(i + 1) * fold_size:], ignore_index=False)

    Features = train.drop('Target', axis=1)
    Target = train['Target']
    w = gradient_descent(Features, Target, 1e-2)
    features = features.append(w, ignore_index=True)
    

    train_pred = reg_prediction(train.drop('Target', axis=1), w)
    R2_train.append(R2(train_pred, train['Target']))
    RMSE_train.append(RMSE(train_pred, train['Target']))

    test_pred = reg_prediction(test.drop('Target', axis=1), w)
    R2_test.append(R2(test_pred, test['Target']))
    RMSE_test.append(RMSE(test_pred, test['Target']))    

# вывод результатов с шагом 0.001
res_df = pd.DataFrame(np.vstack([R2_test, R2_train, RMSE_test, RMSE_train]),
                      index=['R2_test', 'R2_train', 'RMSE_test', 'RMSE_train'])
res_df = res_df.append(features.T)
res_df.columns = ['T1', 'T2', 'T3', 'T4', 'T5']
res_df = pd.concat([res_df, res_df.mean(axis=1).rename('E(mean)'), res_df.std(axis=1).rename('STD')], axis=1)

#print(res_df)
res_df

,T1,T2,T3,T4,T5,E(mean),STD
R2_test,-0.003943,-0.002903,-0.002518,-0.006805,-0.003601,-0.003954,0.001690
R2_train,-0.002371,-0.002425,-0.002309,-0.002372,-0.002310,-0.002357,0.000049
RMSE_test,0.963357,0.952897,0.965782,0.960634,0.964075,0.961349,0.005076
RMSE_train,0.960194,0.962791,0.959478,0.961172,0.960039,0.960735,0.001301
A,0.012984,-0.005349,-0.002935,-0.014414,-0.003542,-0.002651,0.009886
AA,0.009388,-0.017914,0.006571,-0.000902,0.010873,0.001603,0.011815
B,-0.015449,-0.029102,-0.005587,-0.022412,-0.030808,-0.020671,0.010389
BB,-0.007668,0.014227,-0.006494,0.008891,0.009928,0.003777,0.010120
C,-0.038872,-0.008585,0.026128,0.009159,0.007242,-0.000986,0.024489
CC,-0.006719,-0.000526,0.010755,0.017080,-0.013878,0.001343,0.012614
